# 테스트 6회 할 차례

In [9]:
# 크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm

# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
# 입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num + 1
    else:
        return num + 9 * (num - 1)


# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search +"&pd=4"+"&start=" + str(
            start_page)
        print("생성url: ", url)
        return url
    else:
        # url 부분에서 정확도순서로 1일 데이터를 분류 가능
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search +"&pd=4"+"&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls

    # html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)


def news_attrs_crawler(articles, attrs):
    attrs_content = []
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content


# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

# html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    # html 불러오기
    original_html = requests.get(i, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select(
        "div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver, 'href')
    return url


#####뉴스크롤링 시작#####

# 테스트 횟수
num = int(input('몇 번째 테스트? '))
# 검색어 입력
search = input("검색할 키워드를 입력해주세요:")
# 검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):"))  # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ", page, "페이지")
# 검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):"))  # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ", page2, "페이지")

# naver url 생성
url = makeUrl(search, page, page2)

# 뉴스 크롤러 실행
news_titles = []
news_url = []
news_contents = []
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


# 제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist


# 제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

# 1차원 리스트로 만들기(내용 제외)
makeList(news_url_1, news_url)

# NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    # 각 기사 html get하기
    news = requests.get(i, headers=headers)
    news_html = BeautifulSoup(news.text, "html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")

    # 뉴스 본문 가져오기
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')
    
    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one(
            "div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1, repl='', string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ", (page2 + 1 - page) * 10, '개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ', len(news_titles))
print('news_url: ', len(final_urls))
print('news_contents: ', len(news_contents))
print('news_dates: ', len(news_dates))

###데이터 프레임으로 만들기###
import pandas as pd

# 데이터 프레임 만들기
news_df = pd.DataFrame({'date': news_dates, 'title': news_titles})
# news_df = pd.DataFrame({'date': news_dates, 'title': news_titles, 'link': final_urls, 'content': news_contents})

# 중복 행 지우기
news_df = news_df.drop_duplicates(keep='first', ignore_index=True)
print("중복 제거 후 행 개수: ", len(news_df))

# 데이터 프레임 저장
now = datetime.datetime.now()
news_df.to_csv(f'csv/{search} 뉴스원문{num}.csv', encoding='utf-8-sig', index=False)

print(type(news_df['title']))
print(type(news_df['content']))


몇 번째 테스트? 10
검색할 키워드를 입력해주세요:삼성전자

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):20

크롤링할 종료 페이지:  20 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&pd=4&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&pd=4&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&pd=4&start=21', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&pd=4&start=31', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&pd=4&start=41', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&pd=4&start=51', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&pd=4&start=61', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&pd=4&start=71', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&pd=4&start=81', 'https://search.naver.com/search.naver?where=news&

100%|████████████████████████████████████████████████████████████████████████████████| 110/110 [00:14<00:00,  7.43it/s]

검색된 기사 갯수: 총  200 개

[뉴스 제목]
['삼성전자, \'7만전자\' 복귀… 증권가 "주가 더 오른다" 핑크빛 전망', '‘AI 붐’ 엔비디아 깜짝 실적, 삼성전자·SK하이닉스에도 훈풍', '“삼성전자, 역시 믿고 있었다고”...기지개 켜는 반도체 원투펀치, 무슨 일이 [MK위클리반도체]', '회식 중 잃어버린 가방,\xa0삼성 폰으로 찾은 사연 [DD전자상가]', '"韓·이스라엘, 반도체 윈윈 가능 … 삼성전자와 협력 원해"', '엔비디아 CEO “AI는 대만에 황금 기회”…TSMC 들썩, 삼성전자는', "코스피 주간 개인 순매도 1위 '삼성전자'", '셀카찍다 삼성폰 빠트린 印공무원, 물 사흘 퍼냈지만…', '전기료 또 올랐다... 고효율 전자제품으로 갈아탈까?', '주도주 바뀐다…반도체, 2차전지 수익률 역전', '"북미시장 잡아라" TV 사운드바도 삼성 장기집권에 LG 추격전', '오포·화웨이에 시장 뺏기는데 모토로라까지 온다… 中 폴더블폰 반격에 삼성 위기감 고조', "외국산 노트북, 韓 시장 3위 싸움 '치열'", '“이걸 250만원에 누가 사?” 조롱에도 ‘완판’…삼성 큰일났다', '"반도체 협력하자"…한국에 손 내미는 중국의 진짜 속내는?', '스마트폰·TV 수요 위축에…1분기 OLED 패널 출하량 19%↓', '전기료 오르니 ‘절전 가전’ 판매 뜨겁다', '中, 美 반도체 제재에 다급했나...“한국이 공급협력” 일방 발표한 속내', '디리스킹이건 디커플링이건, 우리는 이미 괴롭다', '한국사회를 뒤흔든 양심고백, 삼성의 민낯을 읽다', '"스마트폰 이을 캐시카우"…삼성·애플 맞붙는 시장 어디길래', '"빠트린 삼성폰 찾겠다" 사흘간 저수지 물 210만L 뺀 인도공무원', '"외국인 따라 삼전 살 걸"…이차전지 몰빵 개미들 뒤늦은 후회', '[주간추천주] 美 부채한도 잠정타결…수요회복 기대株 주목', '하반기 폴더블폰 대전...커진 외부화면 앱호환이 성패 가른다', '美, 中의 마이크론 제재에 "성공하지 못할 것"', '“삼전·하닉은

KeyError: 'content'